In [1]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import math as mt
import calendar
import matplotlib.pyplot as plt
import six
from reportlab.pdfgen.canvas import Canvas
from PIL import Image
from PyPDF2 import PdfFileWriter, PdfFileReader, PdfFileMerger
import io
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
import imgkit
import seaborn as sns
import datetime

#### Daily Return Metrics

In [2]:
xls = pd.ExcelFile("2008_2009_constituent_aws_ranked.xlsx")
xls1 = pd.ExcelFile("LBO profile - 19y.xlsm")
positions = pd.read_excel(xls)
returns = pd.read_excel(xls1, sheet = 'Dailies')
positions.drop(0,axis = 0 ,inplace = True)
positions = positions.reset_index()
positions.drop(columns = ['index'], axis = 1, inplace = True)

In [12]:
usr = input("Please choose of the following numbers:\n \
            1. Verify the LBO100 portfolio return for a single day.\n \
            2. Verify the LBO100 portfolio return for a range of dates.\n")
avg = []
ret = []
if(usr == '1'):
    dt = input('Please enter the date in an mm/dd/yyyy format.\n')
    date = pd.to_datetime(dt, format="%m/%d/%Y")
    alist = positions.loc[positions['Periods'] == date]['Returns'].values
    blist = returns.loc[returns['Date'] == date]['LBO (T+1) Return'].values
    if alist.any():
        avg = np.average(alist)
        print("Calculated Portfolio Returns for requested date   :  ", round(avg,2))
    else:
        print("Date not found in the Position dataset.")
    if blist.any():    
        ret = blist[0]
        print("QMIT Reported Portfolio Returns for requested date:  ", round(ret,2))
    else:
        print("Date not found in the Return dataset.")
elif(usr == '2'):
    dt = input('Please enter the start date in an mm/dd/yyyy format.\n')
    dt1 = input('Please enter the end date in an mm/dd/yyyy format.\n')
    date = pd.to_datetime(dt, format="%m/%d/%Y")
    date1 = pd.to_datetime(dt1, format="%m/%d/%Y")
    if(date>date1):
        print("Invalid Dates: End date is before the Start date. Please re-run the cell.")
    else:
        alist = positions.loc[(positions['Periods']>=date) & (positions['Periods']<=date1)]
        blist = returns.loc[(returns['Date']>=date) & (returns['Date']<=date1)]
        if alist['Returns'].values.any():
            for i in range(alist['Periods'].nunique()):
                avg.append(np.average(positions.loc[positions['Periods'] == alist['Periods'].unique()[i]]['Returns'].values))
            for i in range(alist['Periods'].nunique()):
                print("Calculated Portfolio Returns for ",pd.to_datetime(alist['Periods'].unique()[i]), " is: ", round(avg[i],2))
        else:
            print("Date not found in the Position dataset.")
        if blist['LBO (T+1) Return'].values.any():
            for i in range(blist['Date'].nunique()):
                ret.append(returns.loc[returns['Date'] == blist['Date'].unique()[i]]['LBO (T+1) Return'].values)
            for i in range(blist['Date'].nunique()):
                print("QMIT Reported Portfolio Returns for ",pd.to_datetime(blist['Date'].unique()[i]), " is: ", 
                      np.round(ret[i],2)[0])
        else:
            print("Date not found in the Return dataset.")
else:
    print('The number chosen is invalid. Please re-run the cell and select either 1 or 2.')    

Please choose of the following numbers:
             1. Verify the LBO100 portfolio return for a single day.
             2. Verify the LBO100 portfolio return for a range of dates.
2
Please enter the start date in an mm/dd/yyyy format.
11/14/2008
Please enter the end date in an mm/dd/yyyy format.
11/21/2008
Calculated Portfolio Returns for  2008-11-14 00:00:00  is:  -2.17
Calculated Portfolio Returns for  2008-11-17 00:00:00  is:  -1.04
Calculated Portfolio Returns for  2008-11-18 00:00:00  is:  -8.86
Calculated Portfolio Returns for  2008-11-19 00:00:00  is:  -6.85
Calculated Portfolio Returns for  2008-11-20 00:00:00  is:  5.23
Calculated Portfolio Returns for  2008-11-21 00:00:00  is:  8.07
QMIT Reported Portfolio Returns for  2008-11-14 00:00:00  is:  -2.17
QMIT Reported Portfolio Returns for  2008-11-17 00:00:00  is:  -1.04
QMIT Reported Portfolio Returns for  2008-11-18 00:00:00  is:  -8.86
QMIT Reported Portfolio Returns for  2008-11-19 00:00:00  is:  -6.85
QMIT Reported Portfo

#### Performance Metrics

In [13]:
#Annualized Return
temp = []
for i in range(len(returns)):
    temp.append((1+returns['LBO (T+1) Return'].iloc[i]/100))
ret = (np.prod(temp)**(251/len(returns))-1)*100
ret

22.11484060486233

In [14]:
#Annualized Volatility
vol = pd.DataFrame(columns  = ['Value'])
# for i in range (len(returns)):
vol.loc['Vol_B', 'Value'] = returns['LBO (T+1) Return'].std()
vol.loc['Annualized Vol_B','Value'] = vol.loc['Vol_B','Value']*mt.sqrt(251)
vol.loc['Neg Vol_B','Value'] = returns[returns.loc[:,'LBO (T+1) Return']<0].loc[:,'LBO (T+1) Return'].std()
vol.loc['Neg Annualized Vol_B','Value'] = vol.loc['Neg Vol_B','Value']*mt.sqrt(251)
# vol

In [15]:
#Sharpe
sharpe = ret/vol.loc['Annualized Vol_B','Value']
sharpe

0.9303125500997996

In [7]:
#Sortino
sortino = ret/vol.loc['Neg Annualized Vol_B','Value']
sortino

1.30548091627026

In [8]:
#Beta to S&P500
betasp = np.cov(returns['LBO (T+1) Return'], returns['SPX (T+1) Return'])[0][1]/np.cov(returns['LBO (T+1) Return'], returns['SPX (T+1) Return'])[1][1]
betasp

1.0919225850580654

In [9]:
#Max DD
ret = returns['LBO (T+1) Return'].copy()/100 +1
vami = np.multiply.accumulate(ret,0)
end_date = np.argmax((np.maximum.accumulate(vami,0)- vami).values,axis=0)
start_date = vami[:end_date].values.argmax()
mdd = (vami[end_date]-vami[start_date])/vami[start_date]*100
mdd

-27.46578232132179

In [10]:
#Skew & Kurtosis
temps = []
tempk = []
avg = np.average(returns['LBO (T+1) Return'])
std = np.std(returns['LBO (T+1) Return'])
for i in range(len(returns)):
    temps.append((returns['LBO (T+1) Return'].values[i] - avg)**3)  
    tempk.append((returns['LBO (T+1) Return'].values[i] - avg)**4)    
skew = (np.average(temps)/std**3)
kurt = (np.average(tempk)/std**4) - 3
skew,kurt

(-0.08735940556045425, 4.562810020483335)